In [1]:
import torch
from transformers import BertModel, BertTokenizer
import json
import pandas as pd
import gzip
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# Step 1: Load Data from JSON File
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


df = getDF('AMAZON_FASHION_5.json.gz')
df = df[df["reviewText"].apply(lambda x: isinstance(x, str))]

In [4]:
input_texts = df["reviewText"].tolist()
labels = df["overall"].tolist()


In [5]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
encoded_inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)
dataset = TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'], labels)

train_size = int(0.8 * len(dataset))
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])


In [7]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    # Training
    bert_model.train()
    total_train_loss = 0.0
    num_train_batches = 0
    for batch in train_loader:
        input_ids, attention_mask, label = [t.to(device) for t in batch]
        optimizer.zero_grad()
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        num_train_batches += 1
    avg_train_loss = total_train_loss / num_train_batches

    # Evaluation
    bert_model.eval()
    total_eval_loss = 0.0
    num_eval_batches = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids, attention_mask, label = [t.to(device) for t in batch]
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
            total_eval_loss += outputs.loss.item()
            num_eval_batches += 1
    avg_eval_loss = total_eval_loss / num_eval_batches

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Evaluation Loss: {avg_eval_loss:.4f}")

Epoch [1/3], Training Loss: 0.0668, Evaluation Loss: 0.0496
Epoch [2/3], Training Loss: 0.0495, Evaluation Loss: 0.0532
Epoch [3/3], Training Loss: 0.0397, Evaluation Loss: 0.0396


In [15]:
review_text = "I dont like it"

tokenized_review = tokenizer(review_text, padding=True, truncation=True, return_tensors='pt')

input_ids = tokenized_review['input_ids'].to(device)
attention_mask = tokenized_review['attention_mask'].to(device)

with torch.no_grad():
    bert_model.eval()
    outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
    predicted_rating = outputs.logits.item()

print(f"Predicted rating: {predicted_rating:.2f}")

Predicted rating: 2.66
